In [ ]:
# Creating a new ABM using a network structure

In [2]:
# loading packages
import random
import numpy as np
import networkx as nx

import mesa
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.time import RandomActivation
from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer
from mesa.datacollection import DataCollector

# CPS workflow

## World 


## Agents
Universal parameters:
problem_solving
communication

role_correct
role_distribution
role_negotiation lvl3
role_monitoring


Level 1

Level 2

Level 3
role_distributed=1

Level 4
Role_effect=1





## Teams are formed

## Problems
A team is posed with a problem with Task_complexity (1:3) and Size (1000)


Then we distribute role: 1 to len(team) (Do you fulfill your role or not?)
- each is assigned a "correct" role
- Then roles are more probable to become correct based on lvl2-4

- If Lvl 3/4 then roles are distributed correctly/evenly at first
- Self.Role=X

First we try to generate common-ground:
- Based on agents Communication parameter
- Common_ground = 0 or 1

Then we simulate role_alignment:
- There some 0.8 prob that role stays correct
- Theres some 0.2 prob that role might shift to incorrect

Then Solutions are generated:
- generate some 50 to 150 solution (based on problem_solving)
- If lvl doesnt match task lvl then solution cannot be given

If some role > 1:
- Then conflict appears

If common_ground=0
- Lowest solution is chosen

If all roles are evenly distributed and common ground = 1:
    Then choose the 

Adjustement of error/Meta negotiations/Self and team monitoring:
- Resolve_conflict
    - 50% prob of conflict resolvement
    - 90% if lvl 3 is present
- Reach_common_ground
    - 


In [8]:
class TeamAgent(Agent):
    """ An agent with fixed initial wealth."""
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.cps_level = random.randint(1, 4)
        self.problem_solving = random.uniform(0, 1)
        self.communication = random.randint(0, 1)
        self.team_members = []
        self.team_id = None

    def team(self):

        team_size_min, team_size_max = 2, 5

        node = self.unique_id
        neighbor_nodes = self.model.network.get_neighbors(node, include_center=False)
        neighbors = self.model.network.get_cell_list_contents(neighbor_nodes)

        remaining_neighbors = len(neighbors)

        team_members = []

        while remaining_neighbors > 0 and len(team_members) < team_size_max:
            for neighbor in neighbors:
                connection_strength = (self.communication + neighbor.communication) / 2
                if connection_strength > 0.5 and len(team_members) < team_size_max / 2 and len(neighbor.team_members) < team_size_max / 2:
                     if neighbor in [member for team in self.model.teams.values() for member in team]  : 
                        if(len(team_members) < (team_size_max / 2) and len(self.model.teams[neighbor.team_id]) < (team_size_max / 2):
                            self.team_members += neighbor.team_members
                            for member in neighbor.team_members:
                                member.team_id = self.team_id
                            self.model.teams.pop(neighbor.team_id)

                     elif neighbor.team_id < 0:
                        team_members.append(neighbor)
                remaining_neighbors -= 1

            if len(team_members) < team_size_min and self.model.team_id == -1:
                team_id = len(self.model.teams) + 1 
                self.team_id = team_id
                
                self.model.teams[team_id] = team_members
                # print team 
                print("Team", team_id, "formed:", [member.unique_id for member in self.team_members])
    
    def problem(self, team_members):
        # nothing yet
        pass

    def get_team(self):
        return self.team_members
        
    def step(self):
            self.team_members = []
            self.team()
        
        


       

SyntaxError: invalid syntax (1403280782.py, line 28)

In [191]:
# Creating the model
class TeamModel(Model):
    def __init__(self, num_agents):
        self.num_agents = num_agents
        self.schedule = RandomActivation(self)
        
        self.teams = {}
        
        graph = nx.gnm_random_graph(self.num_agents, self.num_agents * 2)
        self.network = mesa.space.NetworkGrid(
            graph)
        
        
        
        for i in range(self.num_agents):
            agent = TeamAgent(i, self)
            self.schedule.add(agent)
            self.network.place_agent(agent, i)

    # A function which checks
    def collect_teams(self):
        # Create a set to keep track of agents that are in teams
        agents_in_teams = set()

        # Print teams for each agent
        for team_id, team in self.teams.items():
            print("Team", team_id, "contains agents:", [agent.unique_id for agent in team])
            agents_in_teams.update(agent.unique_id for agent in team)

        # Check if all agents are distributed and no agent appears in two teams
        all_agents_distributed = len(agents_in_teams) == len(self.schedule.agents)
        no_duplicate_agents = len(agents_in_teams) == len(set(agents_in_teams))

        if all_agents_distributed and no_duplicate_agents:
            print("Evaluation: All agents are distributed in teams of minimum two people, and no team holds the same agent ID twice.")
        else:
            print("Evaluation: Agents are not properly distributed in teams.")

        # Check if any agents are not in a team
        agents_without_team = [agent.unique_id for agent in self.schedule.agents if agent.unique_id not in agents_in_teams]
        if agents_without_team:
            print("Evaluation: The following agents are not in any team:", agents_without_team)

    def step(self):
        self.schedule.step()




    

In [192]:

model = TeamModel(num_agents=20)

for i in range(3):
    model.step()

Team 1 formed: []
Team 2 formed: []


KeyError: 2